# Laboratorio 6 Parte 1

### Reducción de dimensión: Selección de características

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

### Primer integrante:
Nombre: Kevin Martínez Gallego


#### Segundo integrante:

Nombre: Andrés Mauricio Álvarez Ortiz


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de selección de características y usar SVM para resolver un problema de clasificación multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [ ]:
!pip install mlxtend

In [41]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('DB/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

print('Dimensiones de la base de datos de entrenamiento. dim de X: ' + str(np.shape(X)) + '\tdim de Y: ' + str(np.shape(Y)))
print(np.unique(Y, return_counts = True))

Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)
(array([1., 2., 3.]), array([1655,  295,  176], dtype=int64))


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [44]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

def executeSVC():
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
    clf = svm.SVC(decision_function_shape='ovo', kernel='rbf', C = 100, gamma=0.0001)

    #Implemetamos la metodología de validación

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1

    print("\nError de validación sin aplicar SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    #print(('\n\nTiempo total de ejecución: ' + str(time.time()-tiempo_i)) + ' segundos.')
    
    return(str(np.mean(Errores)), str(np.std(Errores)), str(time.time()- tiempo_i))

In [45]:
executeSVC()


Error de validación sin aplicar SFS: 0.07712817787226504 +/- 0.05442325724156325


('0.07712817787226504', '0.05442325724156325', '0.5996193885803223')

## Ejercicio 1

1.1 Describa la metodología de validación que se está aplicando.

R/: Se utiliza validación cruzada (KFold) con 10 subconjuntos.


    
1.2 Con qué modelo se está resolviendo el problema planteado? ¿Cuáles son los parámetros establecidos para el modelo?

R/: Se está empleando el modelo de Máquinas de Vectores de Soporte para clasificación (SVC). Los parámetros son: kernel RBF, parámetro de regularización C = 100 y un ancho de ventana para el kernel gamma = 0.0001

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de selección de características. https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#sequential-feature-selector

In [46]:
#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0)
    
    return sfs

def executeStrategy(strategy, features):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación 

    clf = svm.SVC(decision_function_shape='ovo', kernel='rbf', C = 100, gamma=0.0001)
    
    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    forward = False
    floating = False
    
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        
        if(strategy == 'SFS'):
            forward = True
            floating = False
        elif(strategy == 'SBS'):
            forward = False
            floating = False
        elif(strategy == 'SFFS'):
            forward = True
            floating = True
        elif(strategy == 'SBFS'):
            forward = False
            floating = True
        
        sf = select_features(clf, features, forward, floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1

    #print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    #print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )

    #print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
    
    return(str(np.mean(Errores)), str(np.std(Errores)), str(time.time()- tiempo_i))


In [10]:
executeStrategy('SFFS', 7)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    3.1s finished
Features: 1/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished
Features: 3/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend w

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished
Features: 6/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s finished
Features: 7/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    4.2s finished
Features: 1/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s finished
Features: 6/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.3s finished
Features: 7/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    2.9s finished
Features: 1/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/7[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 


Error de validación aplicando SFS: 0.019911828052138435 +/- 0.002727682121130486

Eficiencia en validación aplicando SFS: 97.91013584117032%


Tiempo total de ejecución: 150.21966457366943 segundos.


('0.019911828052138435', '0.002727682121130486', '150.21966457366943')

## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de características a seleccionar (use 3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS), para que complete la siguiente tabla de resultados:


In [ ]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SBS','SVM + SBS','SVM + SBS','SVM + SFFS','SVM + SFFS','SVM + SFFS','SVM + SBFS','SVM + SBFS','SVM + SBFS']),
    '# de características seleccionadas' : pd.Series([22,3,7,10,3,7,10,3,7,10,3,7,10]),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
#df_types["Error de validación"][8] = "0.019"
#df_types["IC(std)"][8] = "0.002"
#df_types["Tiempo de ejecución"][8] = "107.9 s"

index = 0

acc, std, executionTime = executeSVC()
df_types["Error de validación"][index] = acc
df_types["IC(std)"][index] = std
df_types["Tiempo de ejecución"][index] = executionTime
index += 1

strategies = ['SFS', 'SBS', 'SFFS', 'SBFS']
features = [3,7,10]

for i in range(np.size(strategies)):
    strategy = strategies[i]
    for j in range(np.size(features)):
        acc, std, executionTime = executeStrategy(strategy, features[j])
        df_types["Error de validación"][index] = acc
        df_types["IC(std)"][index] = std
        df_types["Tiempo de ejecución"][index] = executionTime
        index += 1

#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

In [48]:
qgrid_widget.get_changed_df()

Error de validación  \
Tecnica           # de características seleccionadas                         
SVM sin selección 22                                   0.07712817787226504   
SVM + SFS         3                                    0.04311674890112802   
                  7                                   0.019911828052138435   
                  10                                   0.01306550189240313   
SVM + SBS         3                                   0.044214200697968754   
                  7                                    0.01991207385424808   
                  10                                  0.011288625753178627   
SVM + SFFS        3                                    0.04311674890112802   
                  7                                   0.019911828052138435   
                  10                                  0.012804132315821847   
SVM + SBFS        3                                   0.044475734142623125   
                  7                                   0.018866923284069137   
                  10                                   0.01102731079928836   

                                                                    IC(std)  \
Tecnica           # de características seleccionadas                          
SVM sin selección 22                                    0.05442325724156325   
SVM + SFS         3                                    0.005501557850665672   
                  7                                    0.002727682121130486   
                  10                                   0.002347757559935403   
SVM + SBS         3                                    0.004748624592322402   
                  7                                   0.0026168019256843054   
                  10                                   0.001282860778208819   
SVM + SFFS        3                                    0.005501557850665672   
                  7                                    0.002727682121130486   
                  10                                  0.0022432590153498955   
SVM + SBFS        3                                    0.004775170586947885   
                  7                                    0.002543983180019345   
                  10                                  0.0015918971954240022   

                                                     Tiempo de ejecución  
Tecnica           # de características seleccionadas                      
SVM sin selección 22                                  0.6554536819458008  
SVM + SFS         3                                     66.2446219921112  
                  7                                   114.51472759246826  
                  10                                   151.8503873348236  
SVM + SBS         3                                   205.71475911140442  
                  7                                   179.99021291732788  
                  10                                  158.96268939971924  
SVM + SFFS        3                                     90.7502031326294  
                  7                                     157.899822473526  
                  10                                   207.0000627040863  
SVM + SBFS        3                                   440.34995698928833  
                  7                                    328.2050657272339  
                  10                                   266.5297887325287

3.2 Según la teoría vista en el curso, se está usando una función tipo filtro o tipo wrapper y cuál es?

R/: Se está usando una función tipo Wrapper, cuyo modelo de aprendizaje es una Máquina de Soporte Vectorial para clasificación (SVC).

3.3 Con los resultados de la tabla anterior haga un análisis de cuál es el mejor resultado teniendo en cuenta tanto la eficiencia en la clasificación como el costo computacional del modelo y la estrategia implementada.

R/: El mejor resultado fue el obtenido empleando la estrategia SBS y 10 características seleccionadas: Error de validación = 0.0112; IC(std) = 0.0012; Tiempo de ejecución de 158.9 segundos.

3.4 Haga uso del atributo sf.k_feature\_idx\_ (deje evidencia del código usado para esto) para identificar cuáles fueron las características seleccionadas en el mejor de los resultados encontrados. No presente los indices de las características sino sus nombres y descripción.

R/: 

3.5 De acuerdo a los resultados encontrados y la respuesta anterior, usted como ingeniero de datos que le puede sugerir a un médico que esté trabajando en un caso enmarcado dentro del contexto de la base de datos trabajada, para que apoye su diagnóstico?

R/: 